In [2]:
import arcpy
# 输入参数
input_feature = r"C:\Users\Runker\Desktop\ele_sb\gl_merge_data_singles_5_split_curve_repairs_2.shp"
dlmc_field = "DLMC"

# 获取所有唯一的DLMC值
with arcpy.da.SearchCursor(input_feature, [dlmc_field]) as cursor:
    dlmc_values = sorted(set(row[0] for row in cursor))

print(f"发现 {len(dlmc_values)} 个不同的DLMC类型")

发现 8 个不同的DLMC类型


In [4]:
import arcpy
import os
import logging
from datetime import datetime

# 设置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('process.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# 设置工作空间
arcpy.env.workspace = r"D:\ArcGISProjects\workspace\sp2024\MyProject.gdb"
arcpy.env.overwriteOutput = True

def process_land_parcels(input_feature, output_feature, dlmc_field="DLMC",minwidth=6):
    try:
        # 记录开始时间
        start_time = datetime.now()
        logging.info(f"开始处理: {start_time}")
        # 为每个DLMC指定不同的面积阈值
        threshold_values = {
            '其他土地': 50,
            '旱地': 50,
            '林地': 50,
            '水浇地': 50,
            '水田': 50,
            '湿地': 50,
            '种植园用地': 50,
            '草地': 50,
        }
        default_threshold = 50

        # 创建临时工作空间
        temp_workspace = r"D:\ArcGISProjects\workspace\sp2024\temp_workspace.gdb"
        if not arcpy.Exists(temp_workspace):
            arcpy.CreateFileGDB_management(os.path.dirname(arcpy.env.workspace), "temp_workspace")
        
        # 创建第一个临时结果要素类
        temp_result = os.path.join(temp_workspace, "temp_result")
        arcpy.CopyFeatures_management(input_feature, temp_result)
        
        # 获取所有唯一的DLMC值
        with arcpy.da.SearchCursor(input_feature, [dlmc_field]) as cursor:
            dlmc_values = sorted(set(row[0] for row in cursor))
        
        logging.info(f"发现 {len(dlmc_values)} 个不同的DLMC类型")
        
        # 记录处理的总数量
        total_processed = 0
        
        # 为每个DLMC值进行处理
        for index, dlmc_value in enumerate(dlmc_values, 1):
            try:
                logging.info(f"\n正在处理DLMC {index}/{len(dlmc_values)}: {dlmc_value} "
                           f"({index/len(dlmc_values)*100:.1f}%)")
                
                # 创建要素图层
                arcpy.MakeFeatureLayer_management(temp_result, "working_layer")
                
                # 选择当前DLMC值的小于阈值面积的图斑
                where_clause = (f"(Shape_Area < {threshold_values.get(dlmc_value, default_threshold)} OR min_width < {minwidth}) AND {dlmc_field} = '{dlmc_value}'")
                arcpy.SelectLayerByAttribute_management("working_layer", "NEW_SELECTION", where_clause)
                
                # 获取选择的图斑数量
                selected_count = int(arcpy.GetCount_management("working_layer")[0])
                
                if selected_count > 0:
                    logging.info(f"  发现 {selected_count} 个小于 "
                               f"{threshold_values.get(dlmc_value, default_threshold)} 平方米的图斑")
                    
                    # 执行Eliminate操作
                    temp_eliminated = os.path.join(temp_workspace, f"temp_eliminated_{index}")
                    arcpy.Eliminate_management(
                        "working_layer", 
                        temp_eliminated,
                        "LENGTH", 
                        f"{dlmc_field} <> '{dlmc_value}'",  # 保证只合并到相同DLMC类型
                        None
                    )
                    
                    # 更新临时结果
                    arcpy.Delete_management(temp_result)
                    arcpy.CopyFeatures_management(temp_eliminated, temp_result)
                    arcpy.Delete_management(temp_eliminated)
                    
                    total_processed += selected_count
                    logging.info("  已完成消除操作")
                else:
                    logging.info("  未发现需要处理的小图斑")
                
                # 清理临时图层
                arcpy.Delete_management("working_layer")
                
            except Exception as e:
                logging.error(f"处理 {dlmc_value} 时出错: {str(e)}")
                continue
        
        # 将最后的临时结果复制到最终输出要素类
        arcpy.CopyFeatures_management(temp_result, output_feature)
        
        # 清理临时文件
        arcpy.Delete_management(temp_result)
        arcpy.Delete_management(temp_workspace)
        
        # 计算处理时间
        end_time = datetime.now()
        processing_time = end_time - start_time
        
        logging.info(f"\n处理完成:")
        logging.info(f"总共处理了 {total_processed} 个小图斑")
        logging.info(f"最终结果保存在: {os.path.join(arcpy.env.workspace, output_feature)}")
        logging.info(f"总处理时间: {processing_time}")
        
        return True
        
    except Exception as e:
        logging.error(f"处理过程出错: {str(e)}")
        return False

if __name__ == "__main__":
    # 输入参数
    input_feature = r'D:\ArcGISProjects\workspace\sp2024\MyProject.gdb\GL_RESULTS_MultipartToSingle'
    output_feature = 'GL_RESULTS_1'
    
    # 执行处理
    success = process_land_parcels(input_feature, output_feature)
    
    if success:
        logging.info("程序成功完成")
    else:
        logging.error("程序执行失败")

2024-11-18 16:38:19,545 - INFO - 开始处理: 2024-11-18 16:38:19.545777
2024-11-18 16:38:23,572 - INFO - 发现 8 个不同的DLMC类型
2024-11-18 16:38:23,572 - INFO - 
正在处理DLMC 1/8: 其他土地 (12.5%)
2024-11-18 16:38:23,808 - INFO -   发现 60 个小于 50 平方米的图斑
2024-11-18 16:39:49,948 - INFO -   已完成消除操作
2024-11-18 16:39:50,017 - INFO - 
正在处理DLMC 2/8: 旱地 (25.0%)
2024-11-18 16:39:50,221 - INFO -   发现 323 个小于 50 平方米的图斑
2024-11-18 16:41:57,570 - INFO -   已完成消除操作
2024-11-18 16:41:57,641 - INFO - 
正在处理DLMC 3/8: 林地 (37.5%)
2024-11-18 16:41:57,897 - INFO -   发现 958 个小于 200 平方米的图斑
2024-11-18 16:44:39,301 - INFO -   已完成消除操作
2024-11-18 16:44:39,377 - INFO - 
正在处理DLMC 4/8: 水浇地 (50.0%)
2024-11-18 16:44:39,580 - INFO -   未发现需要处理的小图斑
2024-11-18 16:44:39,602 - INFO - 
正在处理DLMC 5/8: 水田 (62.5%)
2024-11-18 16:44:39,799 - INFO -   发现 69 个小于 50 平方米的图斑
2024-11-18 16:46:08,805 - INFO -   已完成消除操作
2024-11-18 16:46:08,882 - INFO - 
正在处理DLMC 6/8: 湿地 (75.0%)
2024-11-18 16:46:09,080 - INFO -   未发现需要处理的小图斑
2024-11-18 16:46:09,112 - INFO - 
正在处理D